In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import hdbscan
from itertools import product
from time import time
import umap
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# 读取结果
df = pd.read_csv("umap_hdbscan_results_beauty.csv")

# 筛选条件：簇数适中 + 噪声不过大
df_filtered = df[
    (df["Clusters"] >= 35)  
]

In [3]:
df_filtered

,nn,md,nc,umap_metric,mcs,ms,eps,hdb_metric,Clusters,Silhouette,NoiseRatio,Score,TimeSec
75,50,0.0,8,euclidean,300,80,0.1,euclidean,35,0.622400,0.264326,0.358074,31.804
76,50,0.0,8,euclidean,300,80,0.2,euclidean,35,0.622400,0.264326,0.358074,32.319
77,50,0.0,8,euclidean,300,80,0.0,euclidean,35,0.622400,0.264326,0.358074,31.867
93,80,0.0,24,euclidean,200,80,0.2,euclidean,44,0.631975,0.277453,0.354522,41.778
100,120,0.0,16,euclidean,300,50,0.0,euclidean,37,0.622014,0.268027,0.353987,47.047
...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,50,0.1,16,euclidean,200,50,0.0,euclidean,49,0.564759,0.362635,0.202123,27.846
722,50,0.1,16,euclidean,200,50,0.2,euclidean,49,0.564759,0.362635,0.202123,29.029
726,50,0.1,16,euclidean,200,80,0.1,euclidean,43,0.494311,0.322715,0.171596,31.836
727,50,0.1,16,euclidean,200,80,0.2,euclidean,43,0.494311,0.322715,0.171596,31.144


In [4]:
# 按 Score 排序，取 Top 10
df_top = df_filtered.sort_values(by="Silhouette", ascending=False)

In [5]:
df_top.head(20)

,nn,md,nc,umap_metric,mcs,ms,eps,hdb_metric,Clusters,Silhouette,NoiseRatio,Score,TimeSec
176,50,0.0,16,euclidean,200,120,0.2,euclidean,44,0.652880,0.319032,0.333849,33.078
187,50,0.0,16,euclidean,200,120,0.0,euclidean,45,0.652425,0.321842,0.330583,32.368
188,50,0.0,16,euclidean,200,120,0.1,euclidean,45,0.652425,0.321842,0.330583,31.035
153,120,0.0,8,euclidean,300,120,0.1,euclidean,37,0.642131,0.303200,0.338932,36.098
152,120,0.0,8,euclidean,300,120,0.0,euclidean,37,0.642131,0.303200,0.338932,37.925
131,120,0.0,8,euclidean,300,120,0.2,euclidean,35,0.641220,0.296776,0.344444,40.000
147,80,0.0,24,euclidean,200,120,0.2,euclidean,42,0.640498,0.300215,0.340283,40.716
105,80,0.0,8,euclidean,300,80,0.2,euclidean,36,0.640247,0.287018,0.353229,34.190
107,80,0.0,8,euclidean,300,80,0.0,euclidean,37,0.639889,0.289078,0.350811,33.191
108,80,0.0,8,euclidean,300,80,0.1,euclidean,37,0.639889,0.289078,0.350811,34.801
